In [1]:
spark

Waiting for a Spark session to start...

Spark application_1540458187951_76578: Some(http://gw02.itversity.com:4041)

In [2]:
sc.getConf.getAll.filter(_._2.contains("/proxy/"))(0)._2

Waiting for a Spark session to start...

http://rm01.itversity.com:19288/proxy/application_1540458187951_76578

In [3]:
def getType(o: Any) = o.getClass.getCanonicalName

getType: (o: Any)String


In [4]:
val os_name = System.getProperty("os.name")
val hdfs_home = "/user/" + System.getenv("HOME").split("/")(2)

os_name = Linux
hdfs_home = /user/kranthidr


/user/kranthidr

In [5]:
val path = hdfs_home+"/dataSets/spark-guide/activity-data/"

path = /user/kranthidr/dataSets/spark-guide/activity-data/


/user/kranthidr/dataSets/spark-guide/activity-data/

In [6]:
// in Scala
val static = spark.read.json(path)

static = [Arrival_Time: bigint, Creation_Time: bigint ... 8 more fields]


[Arrival_Time: bigint, Creation_Time: bigint ... 8 more fields]

In [7]:
val streaming = spark
  .readStream
  .schema(static.schema)
  .option("maxFilesPerTrigger", 10)
  .json(path)
  .groupBy("gt")
  .count()

streaming = [gt: string, count: bigint]


[gt: string, count: bigint]

In [8]:
val query = streaming
  .writeStream
  .outputMode("complete")
  .option("checkpointLocation", "/tmp/")
  .queryName("test_stream")
  .format("memory")
  .start()

query = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@170956c3


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@170956c3

In [9]:
// COMMAND ----------

query.status

{
  "message" : "Getting offsets from FileStreamSource[hdfs://nn01.itversity.com:8020/user/kranthidr/dataSets/spark-guide/activity-data]",
  "isDataAvailable" : false,
  "isTriggerActive" : true
}

In [10]:
// COMMAND ----------

query.recentProgress

Array({
  "id" : "e69c04b1-694d-42fe-b858-3d3e0f253a00",
  "runId" : "d5422c77-3e88-4fd3-8632-8d9308cc66f2",
  "name" : "test_stream",
  "timestamp" : "2019-03-30T09:35:09.946Z",
  "batchId" : 8,
  "numInputRows" : 0,
  "processedRowsPerSecond" : 0.0,
  "durationMs" : {
    "getOffset" : 115,
    "triggerExecution" : 295
  },
  "stateOperators" : [ ],
  "sources" : [ {
    "description" : "FileStreamSource[hdfs://nn01.itversity.com:8020/user/kranthidr/dataSets/spark-guide/activity-data]",
    "startOffset" : {
      "logOffset" : 7
    },
    "endOffset" : {
      "logOffset" : 7
    },
    "numInputRows" : 0,
    "processedRowsPerSecond" : 0.0
  } ],
  "sink" : {
    "description" : "MemorySink"
  }
})


[{
  "id" : "e69c04b1-694d-42fe-b858-3d3e0f253a00",
  "runId" : "d5422c77-3e88-4fd3-8632-8d9308cc66f2",
  "name" : "test_stream",
  "timestamp" : "2019-03-30T09:35:09.946Z",
  "batchId" : 8,
  "numInputRows" : 0,
  "processedRowsPerSecond" : 0.0,
  "durationMs" : {
    "getOffset" : 115,
    "triggerExecution" : 295
  },
  "stateOperators" : [ ],
  "sources" : [ {
    "description" : "FileStreamSource[hdfs://nn01.itversity.com:8020/user/kranthidr/dataSets/spark-guide/activity-data]",
    "startOffset" : {
      "logOffset" : 7
    },
    "endOffset" : {
      "logOffset" : 7
    },
    "numInputRows" : 0,
    "processedRowsPerSecond" : 0.0
  } ],
  "sink" : {
    "description" : "MemorySink"
  }
}]

In [25]:
// COMMAND ----------

//val spark: SparkSession = ...
import org.apache.spark.sql.streaming.StreamingQueryListener
import org.apache.spark.sql.streaming.StreamingQueryListener._

In [26]:
spark.streams.addListener(new StreamingQueryListener() {
    override def onQueryStarted(queryStarted: QueryStartedEvent): Unit = {
        println("Query started: " + queryStarted.id)
    }
    override def onQueryTerminated(
      queryTerminated: QueryTerminatedEvent): Unit = {
        println("Query terminated: " + queryTerminated.id)
    }
    override def onQueryProgress(queryProgress: QueryProgressEvent): Unit = {
        println("Query made progress: " + queryProgress.progress)
    }
})

In [28]:
// COMMAND ----------

// import org.apache.spark.sql.streaming.StreamingQueryListener

// // import org.apache.kafka.clients.producer.KafkaProducer
// // Message: <console>:16: error: object kafka is not a member of package org.apache
// //        import org.apache.kafka.clients.producer.KafkaProducer

// class KafkaMetrics(servers: String) extends StreamingQueryListener {
    
//   val kafkaProperties = new Properties()
//   kafkaProperties.put(
//     "bootstrap.servers",
//     servers)
//   kafkaProperties.put(
//     "key.serializer",
//     "kafkashaded.org.apache.kafka.common.serialization.StringSerializer")
//   kafkaProperties.put(
//     "value.serializer",
//     "kafkashaded.org.apache.kafka.common.serialization.StringSerializer")

//   val producer = new KafkaProducer[String, String](kafkaProperties)

//   override def onQueryProgress(event:
//     StreamingQueryListener.QueryProgressEvent): Unit = {
//     producer.send(new ProducerRecord("streaming-metrics",
//       event.progress.json))
//   }
//   override def onQueryStarted(event:
//     StreamingQueryListener.QueryStartedEvent): Unit = {}
//   override def onQueryTerminated(event:
//     StreamingQueryListener.QueryTerminatedEvent): Unit = {}
// }



// // COMMAND ----------

Name: Syntax Error.
Message: 
StackTrace: 